Running this notebook will webscrape the data from the internet which might take a long time.
Don't need to run if data is already available.

In [ ]:
import requests
import os
import shutil
import time
import pandas as pd

In [ ]:
!pip install selenium

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [ ]:
# Run the chrome driver
service = Service(executable_path='./chromedriver.exe')
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [ ]:
# Select range of years for our dataset to be webscraped
years = list(range(1991,2025))

In [ ]:
# First, get the MVPs data
# This method is to be able to read players with special characters (e.g. Nikola Jokić)
url_start = "https://www.basketball-reference.com/awards/awards_{}.html#mvp"
for year in years:
    url = url_start.format(year)

    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)") # Executes Javascript that scrolls to bottom of page
    time.sleep(4) # Sleep timeout VERY IMPORTANT to not get rate limited by the server and avoid timeout jail, 20 pages/min is the limit

    with open("mvp/{}.html".format(year), "w+", encoding = "utf-8-sig") as f: # utf-8-sig to allow special characters (e.g. Nikola Jokić)
        f.write(driver.page_source) # Write it to mvp/1991.html, mvp/1992.html, and so on in the mvp folder

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# Parsing, use 2020 as the example first:
# Open one of the htmls to inspect the contents
with open("mvp/2020.html", encoding = "utf-8-sig") as f:
    page = f.read()

# Delete the overheader html row, not needed
soup = BeautifulSoup(page, 'html.parser')
soup.find('tr', class_="over_header").decompose()

In [ ]:
# Only need the mvp table, other tables not needed
mvp_table = soup.find_all(id="mvp")[0]

In [ ]:
# Let pandas read the html
mvp_2020 = pd.read_html(str(mvp_table))[0]

C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14124\3631753777.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp_2020 = pd.read_html(str(mvp_table))[0]


In [ ]:
# Make sure players with special characters are correctly typed, e.g: Luka Dončić, Nikola Jokić
mvp_2020.head(15)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Giannis Antetokounmpo,25,MIL,85,962,1010,0.952,63,30.4,29.5,13.6,5.6,1.0,1.0,0.553,0.304,0.633,11.1,0.279
1,2,LeBron James,35,LAL,16,753,1010,0.746,67,34.6,25.3,7.8,10.2,1.2,0.5,0.493,0.348,0.693,9.8,0.204
2,3,James Harden,30,HOU,0,367,1010,0.363,68,36.5,34.3,6.6,7.5,1.8,0.9,0.444,0.355,0.865,13.1,0.254
3,4,Luka Dončić,20,DAL,0,200,1010,0.198,61,33.6,28.8,9.4,8.8,1.0,0.2,0.463,0.316,0.758,8.8,0.207
4,5,Kawhi Leonard,28,LAC,0,168,1010,0.166,57,32.4,27.1,7.1,4.9,1.8,0.6,0.470,0.378,0.886,8.7,0.226
5,6,Anthony Davis,26,LAL,0,82,1010,0.081,62,34.4,26.1,9.3,3.2,1.5,2.3,0.503,0.330,0.846,11.1,0.250
6,7,Chris Paul,34,OKC,0,26,1010,0.026,70,31.5,17.6,5.0,6.7,1.6,0.2,0.489,0.365,0.907,8.9,0.193
7,8,Damian Lillard,29,POR,0,23,1010,0.023,66,37.5,30.0,4.3,8.0,1.1,0.3,0.463,0.401,0.888,11.6,0.225
8,9,Nikola Jokić,24,DEN,0,18,1010,0.018,73,32.0,19.9,9.7,7.0,1.2,0.6,0.528,0.314,0.817,9.8,0.202
9,10,Pascal Siakam,25,TOR,0,17,1010,0.017,60,35.2,22.9,7.3,3.5,1.0,0.9,0.453,0.359,0.792,5.4,0.123


In [ ]:
# We will combine all the data into one mvp table, so need to put a new column indicating year of mvp
mvp_2020["Year"] = 2020

In [ ]:
# Final result for 2020 MVPs
mvp_2020.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Giannis Antetokounmpo,25,MIL,85,962,1010,0.952,63,30.4,...,13.6,5.6,1.0,1.0,0.553,0.304,0.633,11.1,0.279,2020
1,2,LeBron James,35,LAL,16,753,1010,0.746,67,34.6,...,7.8,10.2,1.2,0.5,0.493,0.348,0.693,9.8,0.204,2020
2,3,James Harden,30,HOU,0,367,1010,0.363,68,36.5,...,6.6,7.5,1.8,0.9,0.444,0.355,0.865,13.1,0.254,2020
3,4,Luka Dončić,20,DAL,0,200,1010,0.198,61,33.6,...,9.4,8.8,1.0,0.2,0.463,0.316,0.758,8.8,0.207,2020
4,5,Kawhi Leonard,28,LAC,0,168,1010,0.166,57,32.4,...,7.1,4.9,1.8,0.6,0.470,0.378,0.886,8.7,0.226,2020


In [ ]:
# Now repeat the same thing for all years 1991-2025
dfs = []
for year in years:
    with open("mvp/{}.html".format(year), encoding = "utf-8-sig") as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(str(mvp_table))[0]
    mvp_df["Year"] = year
    dfs.append(mvp_df)

C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\875247043.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp_df = pd.read_html(str(mvp_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\875247043.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp_df = pd.read_html(str(mvp_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\875247043.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  mvp_df = pd.read_html(str(mvp_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\875247043.py:10: FutureWarning: Passing literal html to 'read_html' is deprecated and wil

In [ ]:
# Concatenate the lists of dataframes into a single one
mvps = pd.concat(dfs)

In [ ]:
mvps.tail(15)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
7,8,Luka Dončić,23,DAL,0,10,1000,0.010,66,36.2,...,8.6,8.0,1.4,0.5,0.496,0.342,0.742,10.2,0.204,2023
8,9,Stephen Curry,34,GSW,0,5,1000,0.005,56,34.7,...,6.1,6.3,0.9,0.4,0.493,0.427,0.915,7.8,0.192,2023
9,10,Jimmy Butler,33,MIA,0,3,1000,0.003,64,33.4,...,5.9,5.3,1.8,0.3,0.539,0.350,0.850,12.3,0.277,2023
10,11,De'Aaron Fox,25,SAC,0,2,1000,0.002,73,33.4,...,4.2,6.1,1.1,0.3,0.512,0.324,0.780,7.4,0.146,2023
11,12T,Jalen Brunson,26,NYK,0,1,1000,0.001,68,35.0,...,3.5,6.2,0.9,0.2,0.491,0.416,0.829,8.7,0.175,2023
12,12T,Ja Morant,23,MEM,0,1,1000,0.001,61,31.9,...,5.9,8.1,1.1,0.3,0.466,0.307,0.748,6.0,0.148,2023
0,1,Nikola Jokić,28,DEN,79,926,990,0.935,79,34.6,...,12.4,9.0,1.4,0.9,0.583,0.359,0.817,17.0,0.299,2024
1,2,Shai Gilgeous-Alexander,25,OKC,15,640,990,0.646,75,34.0,...,5.5,6.2,2.0,0.9,0.535,0.353,0.874,14.6,0.275,2024
2,3,Luka Dončić,24,DAL,4,566,990,0.572,70,37.5,...,9.2,9.8,1.4,0.5,0.487,0.382,0.786,12.0,0.220,2024
3,4,Giannis Antetokounmpo,29,MIL,1,192,990,0.194,73,35.2,...,11.5,6.5,1.2,1.1,0.611,0.274,0.657,13.2,0.246,2024


In [ ]:
# Convert to csv
mvps.to_csv("mvps.csv")

In [ ]:
# Now for player stats
# This method is to be able to run javascript that is used in the website to render all of the rows after the page is accessed.
# Not using this method will result in only ~20 player rows instead of the full 500+ players in the NBA.
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

for year in years:
    url = player_stats_url.format(year)

    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)") # Javascript to scroll and reveal rows that are otherwise not rendered in the page
    time.sleep(4)

    with open("player/{}.html".format(year), "w+", encoding = "utf-8-sig") as f:
        f.write(driver.page_source)

In [ ]:
dfs = []
for year in years:
    with open("player/{}.html".format(year), encoding = "utf-8-sig") as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\4049039344.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\4049039344.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\4049039344.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_df = pd.read_html(str(player_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\4049039344.py:9: FutureWarning: Passing literal html to 'read_html' is 

In [ ]:
players = pd.concat(dfs)

In [ ]:
# Make sure all of the 500+ playersare correctly stored
players[players["Year"] == 2024]

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
0,1,Joel Embiid,29,PHI,C,39,39,33.6,11.5,21.8,...,8.6,11.0,5.6,1.2,1.7,3.8,2.9,34.7,AS,2024
1,2,Luka Dončić,24,DAL,PG,70,70,37.5,11.5,23.6,...,8.4,9.2,9.8,1.4,0.5,4.0,2.1,33.9,"MVP-3,CPOY-6,AS,NBA1",2024
2,3,Giannis Antetokounmpo,29,MIL,PF,73,73,35.2,11.5,18.8,...,8.8,11.5,6.5,1.2,1.1,3.4,2.9,30.4,"MVP-4,DPOY-9,CPOY-12,AS,NBA1",2024
3,4,Shai Gilgeous-Alexander,25,OKC,PG,75,75,34.0,10.6,19.8,...,4.7,5.5,6.2,2.0,0.9,2.2,2.5,30.1,"MVP-2,DPOY-7,CPOY-3,AS,NBA1",2024
4,5,Jalen Brunson,27,NYK,PG,77,77,35.4,10.3,21.4,...,3.1,3.6,6.7,0.9,0.2,2.4,1.9,28.7,"MVP-5,CPOY-5,AS,NBA2",2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,569,Ron Harper Jr.,23,TOR,PF,1,0,4.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,NaN,2024
755,570,Justin Jackson,28,MIN,SF,2,0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2024
756,571,Dmytro Skapintsev,25,NYK,C,2,0,1.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2024
757,572,Javonte Smart,24,PHI,PG,1,0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2024


In [ ]:
# Make sure players with special characters are correctly typed
players[players["Year"] == 2022].head(10)

,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Awards,Year
0,1,Joel Embiid,27,PHI,C,68,68,33.8,9.8,19.6,...,9.6,11.7,4.2,1.1,1.5,3.1,2.7,30.6,"MVP-2,AS,NBA2",2022
1,2,LeBron James,37,LAL,C,56,56,37.2,11.4,21.8,...,7.1,8.2,6.2,1.3,1.1,3.5,2.2,30.3,"MVP-10,AS,NBA3",2022
2,3,Giannis Antetokounmpo,27,MIL,PF,67,67,32.9,10.3,18.6,...,9.6,11.6,5.8,1.1,1.4,3.3,3.2,29.9,"MVP-3,DPOY-6,AS,NBA1",2022
3,4,Kevin Durant,33,BRK,PF,55,55,37.2,10.5,20.3,...,6.9,7.4,6.4,0.9,0.9,3.5,2.1,29.9,"MVP-10,AS,NBA2",2022
4,5,Luka Dončić,22,DAL,PG,65,65,35.4,9.9,21.6,...,8.3,9.1,8.7,1.2,0.6,4.5,2.2,28.4,"MVP-5,AS,NBA1",2022
5,6,Trae Young,23,ATL,PG,76,76,34.9,9.4,20.3,...,3.1,3.7,9.7,0.9,0.1,4.0,1.7,28.4,"AS,NBA3",2022
6,7,DeMar DeRozan,32,CHI,PF,76,76,36.1,10.2,20.2,...,4.4,5.2,4.9,0.9,0.3,2.4,2.3,27.9,"MVP-10,AS,NBA2",2022
7,8,Kyrie Irving,29,BRK,PG,29,29,37.6,10.0,21.2,...,3.8,4.4,5.8,1.4,0.6,2.5,2.8,27.4,NaN,2022
8,9,Ja Morant,22,MEM,PG,57,57,33.1,10.2,20.6,...,4.4,5.7,6.7,1.2,0.4,3.4,1.5,27.4,"MVP-7,MIP-1,AS,NBA2",2022
9,10,Nikola Jokić,26,DEN,C,74,74,33.5,10.3,17.7,...,11.0,13.8,7.9,1.5,0.9,3.8,2.6,27.1,"MVP-1,AS,NBA1",2022


In [ ]:
# Convert to csv
players.to_csv("players.csv")

In [ ]:
years = list(range(1991,2025))

In [ ]:
# Now for ADVANCED player stats
# Contains advanced statistics of players that will be very useful predictors to us
# Examples: Player Efficiency Rating (PER), Win Shares (WR), Box Plus Minus (BPM), etc.
player_advanced_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_advanced.html"

for year in years:
    url = player_advanced_stats_url.format(year)

    driver.get(url)
    driver.execute_script("window.scrollTo(1,10000)") # Javascript to scroll and reveal rows that are otherwise not rendered in the page
    time.sleep(4)

    with open("player_advanced/{}.html".format(year), "w+", encoding = "utf-8-sig") as f:
        f.write(driver.page_source)

In [ ]:
dfs = []
for year in years:
    with open("player_advanced/{}.html".format(year), encoding = "utf-8-sig") as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    player_advanced_table = soup.find_all(id="advanced")[0]
    player_advanced_df = pd.read_html(str(player_advanced_table))[0]
    player_advanced_df["Year"] = year
    dfs.append(player_advanced_df)

C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\600107487.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_advanced_df = pd.read_html(str(player_advanced_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\600107487.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_advanced_df = pd.read_html(str(player_advanced_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\600107487.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_advanced_df = pd.read_html(str(player_advanced_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\600107487.py:9: Futu

In [ ]:
players_advanced = pd.concat(dfs)

In [ ]:
players_advanced

,Rk,Player,Age,Team,Pos,G,GS,MP,PER,TS%,...,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Awards,Year
0,1,Chris Mullin,27,GSW,SF,82,82,3315,21.4,.618,...,9.7,2.4,12.2,.176,4.5,0.1,4.7,5.6,"AS,NBA2",1991
1,2,Karl Malone,27,UTA,PF,82,82,3302,24.8,.596,...,9.9,5.6,15.5,.225,4.8,0.6,5.4,6.2,"MVP-5,AS,NBA1",1991
2,3,Tim Hardaway,24,GSW,PG,82,82,3215,20.9,.547,...,7.4,2.5,9.9,.148,4.5,-0.1,4.4,5.2,"MVP-19,AS",1991
3,4,Tyrone Corbin,28,MIN,SF,82,82,3196,16.6,.499,...,2.6,2.9,5.4,.081,0.5,0.4,0.9,2.3,NaN,1991
4,5,Pooh Richardson,24,MIN,PG,82,82,3154,18.1,.492,...,4.5,1.3,5.9,.089,3.1,-1.1,2.0,3.2,NaN,1991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
754,569,Jalen Crutcher,24,NOP,PG,1,0,3,-12.6,.000,...,0.0,0.0,0.0,-0.334,-18.5,-7.8,-26.2,0.0,NaN,2024
755,570,Dmytro Skapintsev,25,NYK,C,2,0,2,-19.3,.000,...,0.0,0.0,0.0,-0.483,-16.0,-9.8,-25.9,0.0,NaN,2024
756,571,Justin Jackson,28,MIN,SF,2,0,1,0.0,NaN,...,0.0,0.0,0.0,.031,-6.3,-1.2,-7.5,0.0,NaN,2024
757,572,Javonte Smart,24,PHI,PG,1,0,1,0.0,NaN,...,0.0,0.0,0.0,.005,-6.2,-2.1,-8.3,0.0,NaN,2024


In [ ]:
players_advanced.to_csv("players_advanced.csv")

In [ ]:
# Now for team stats
# This part does not need the selenium chromedriver/javascript method like before because:
# 1. No team name have special characters, and
# 2. The html page does not have rows rendered with javascript only after loading the page

team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"
for year in years:
    url = team_stats_url.format(year)

    data = requests.get(url)
    time.sleep(4)

    with open("team/{}.html".format(year), "w+", encoding = "utf-8-sig") as f:
        f.write(data.text)

In [ ]:
# Parsing the data
dfs = []
for year in years:
    with open("team/{}.html".format(year), encoding = "utf-8-sig") as f:
        page = f.read()

    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()

    #for eastern conference
    e_table = soup.find_all(id="divs_standings_E")[0] # Find all eastern conference tables
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year # Add the year column
    e_df["Team"] = e_df["Eastern Conference"] # Since we later combine both east and west, need to retain the conference under the same column name when merging the tables
    del e_df["Eastern Conference"] # Because we have the "Team" column to standardize, no need for the "Eastern Conference" column
    dfs.append(e_df) # Append to the dataframe

    #for western conference
    w_table = soup.find_all(id="divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    dfs.append(w_df)

C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\402537560.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  e_df = pd.read_html(str(e_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\402537560.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  w_df = pd.read_html(str(w_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\402537560.py:12: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  e_df = pd.read_html(str(e_table))[0]
C:\Users\s1155147304\AppData\Local\Temp\ipykernel_14628\402537560.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed

In [ ]:
# Concatenate across all years
teams = pd.concat(dfs)

In [ ]:
# The 'Division' rows will need to be cleaned later on
teams.head(15)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets
5,24,58,.293,32.0,101.8,107.8,-5.91,1991,Miami Heat
6,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,Central Division,1991,Central Division
7,61,21,.744,—,110.0,101.0,8.57,1991,Chicago Bulls*
8,50,32,.610,11.0,100.1,96.8,3.08,1991,Detroit Pistons*
9,48,34,.585,13.0,106.4,104.0,2.33,1991,Milwaukee Bucks*


In [ ]:
teams.tail(15)

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
3,56,26,.683,1.0,113.0,106.5,6.39,2024,Minnesota Timberwolves*
4,31,51,.378,26.0,115.7,120.5,-4.22,2024,Utah Jazz
5,21,61,.256,36.0,106.4,115.4,-8.29,2024,Portland Trail Blazers
6,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,Pacific Division,2024,Pacific Division
7,51,31,.622,—,115.6,112.3,3.41,2024,Los Angeles Clippers*
8,49,33,.598,2.0,116.2,113.2,3.08,2024,Phoenix Suns*
9,47,35,.573,4.0,118.0,117.4,1.07,2024,Los Angeles Lakers*
10,46,36,.561,5.0,116.6,114.8,2.29,2024,Sacramento Kings
11,46,36,.561,5.0,117.8,115.2,2.77,2024,Golden State Warriors
12,Southwest Division,Southwest Division,Southwest Division,Southwest Division,Southwest Division,Southwest Division,Southwest Division,2024,Southwest Division


In [ ]:
teams.to_csv("teams.csv")